## imports

In [1]:
{-# LANGUAGE PackageImports #-}
{-# LANGUAGE OverloadedStrings #-}
import "atidot-anonymizer" Atidot.Anonymizer.Monad
import qualified "atidot-anonymizer" Atidot.Anonymizer.Run as Anonymizer (runExample)
import "atidot-anonymizer" Atidot.Anonymizer.XML
import "atidot-anonymizer" Atidot.Anonymizer.Types
import "atidot-anonymizer" Atidot.Anonymizer.Utils
import "mtl"          Control.Monad.State (evalStateT)
import "data-default" Data.Default (def)

import Control.Monad
import qualified Data.Text as T
import qualified Data.ByteString.Lazy as BL
import qualified "bytestring" Data.ByteString.Lazy.Char8 as BL8


## Contants

In [2]:
anonymizationPath :: Path
anonymizationPath = ["/","a","b","c"]

xmlPath = "example.csv"

## Example XML:
let's print the example xml used in this notebook

In [3]:
xmlData <- BL.readFile xmlPath
BL8.putStrLn xmlData

a,b,c
foo,bar,doo
boo,gar,joo
goo,far,koo

## Example 1: print all paths in file

In [4]:
script :: Anonymizer [Path]
script = do
    whileNext $ return ()
    paths

execute `runExample` from Atidot.Anonymizer.Run, that prints two things:
1. XML after the script's traversal on the file
1. The paths traversed by the script

In [5]:
Anonymizer.runExample xmlPath script

hello from csv
[[("a","foo"),("a","boo"),("a","goo")],[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
bobo
Zip [] [[("a","foo"),("a","boo"),("a","goo")],[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
Zip [[("a","foo"),("a","boo"),("a","goo")]] [[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
bobo
Zip [[("a","foo"),("a","boo"),("a","goo")]] [[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
Zip [[("b","bar"),("b","gar"),("b","far")],[("a","foo"),("a","boo"),("a","goo")]] [[("c","doo"),("c","joo"),("c","koo")]]

[["a"],["b"],["c"]]

## Example 2: hash explicit paths

In [6]:
script2 :: Anonymizer ()
script2 = do
    whileNext $ do
        path <- current
        return ()
        --when (path == anonymizationPath) hash
    return ()

In [7]:
Anonymizer.runExample xmlPath script2

hello from csv
[[("a","foo"),("a","boo"),("a","goo")],[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
bobo
Zip [] [[("a","foo"),("a","boo"),("a","goo")],[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
Zip [[("a","foo"),("a","boo"),("a","goo")]] [[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
bobo
Zip [[("a","foo"),("a","boo"),("a","goo")]] [[("b","bar"),("b","gar"),("b","far")],[("c","doo"),("c","joo"),("c","koo")]]
Zip [[("b","bar"),("b","gar"),("b","far")],[("a","foo"),("a","boo"),("a","goo")]] [[("c","doo"),("c","joo"),("c","koo")]]

## Example 3: hash value of b element and it's subtree

Hashes all elements in subtree

In [11]:
script3 :: Anonymizer ()
script3 = do
    whileNext $ do
        path <- current
        when (path =~ ["b"]) hash

In [12]:
Anonymizer.runExample xmlPath script3

Data/List/Zipper.hs:65:1-24: Non-exhaustive patterns in function cursor

## Example4: hash c or d elemets

Hashes all paths containing the Substrings `c` or `d`

In [18]:
script4 :: Anonymizer ()
script4 = do
    whileNext $ do
        path <- current
        when (path =~ ["c"] || path =~ ["d"]) hash
        

In [20]:
Anonymizer.runExample xmlPath script4

<?xml version="1.0" encoding="UTF-8"?>
<a>
  <b>
    <c>4b093efdc14b01cdf75efcb08dcfd3f1afdfc567f9c60d338d3e8a941f4051bb</c>
    <d>2ac33784352a5ec322f8cdeebc8199ea41f3de2ecb30cd8bc48e19b9455f81a4</d>
  </b>
  <e>
    <f>
      "goo"
    </f>
    <d>2e8dffc6d0deb87ad1880b9d4700f3c50643e58093d05f2248a39a89889446e7</d>
  </e>
</a>